In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### Import the whole dataset

In [3]:
# Dataset load
cars = pd.read_csv("used_cars_data.csv")

# Dataset overview
cars.head(2)

/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,engine_type,exterior_color,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,interior_color,isCab,is_certified,is_cpo,is_new,is_oemcpo,latitude,length,listed_date,listing_color,listing_id,longitude,main_picture_url,major_options,make_name,maximum_seating,mileage,model_name,owner_count,power,price,salvage,savings_amount,seller_rating,sp_id,sp_name,theft_title,torque,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,522,960,[!@@Additional Info@@!]Engine: 2.4L I4 ZERO EV...,I4,1300.0,I4,Solar Yellow,NaN,NaN,True,Jeep,41.2 in,12.7 gal,Gasoline,NaN,66.5 in,NaN,177.0,Black,NaN,NaN,NaN,True,NaN,18.3988,166.6 in,2019-04-06,YELLOW,237132766,-66.1582,https://static.cargurus.com/images/forsale/202...,['Quick Order Package'],Jeep,5 seats,7.0,Renegade,NaN,"177 hp @ 5,750 RPM",23141.0,NaN,0,2.8,370599.0,Flagship Chrysler,NaN,"200 lb-ft @ 1,750 RPM",A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,207,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",I4,2000.0,I4,Narvik Black,NaN,NaN,True,Land Rover,39.1 in,17.7 gal,Gasoline,NaN,68 in,NaN,246.0,Black (Ebony),NaN,NaN,NaN,True,NaN,18.4439,181 in,2020-02-15,BLACK,265946296,-66.0785,https://static.cargurus.com/images/forsale/202...,['Adaptive Cruise Control'],Land Rover,7 seats,8.0,Discovery Sport,NaN,"246 hp @ 5,500 RPM",46500.0,NaN,0,3.0,389227.0,Land Rover San Juan,NaN,"269 lb-ft @ 1,400 RPM",A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


### Take the first 10000 entries for test

In [4]:
cars_mini = cars.head(10000)

### First create depreciation_info, pickup_truck table

In [79]:
Dp = cars_mini[['frame_damaged', 'has_accidents', 'salvage', 'isCab', 'theft_title']]
# df_new = df.drop(df[ <condition1> & <condition2> ].index)
Dp.drop(Dp[(Dp['frame_damaged'] == False) & 
           (Dp['has_accidents'] == False) & 
           (Dp['salvage'] == False) & 
           (Dp['isCab'] == False) & 
           (Dp['theft_title'] == False)].index, inplace=True)
Dp.dropna(how='all', inplace=True)
Dp.sample(10)
# Dp.shape

/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,frame_damaged,has_accidents,salvage,isCab,theft_title
7688,True,False,False,False,False
4879,False,False,False,True,False
2529,False,True,False,False,False
243,False,True,False,False,False
2007,False,True,False,False,False
9133,False,False,False,True,False
4018,False,False,False,True,False
5190,False,False,False,False,True
5409,False,False,False,False,True
2965,False,True,True,False,False


In [80]:
pk = cars_mini[['bed', 'bed_length', 'cabin']]
pk['bed_length'] = pk['bed_length'].replace(r'--', np.nan, regex=True)
pk['bed_length'] = pk['bed_length'].str.replace(r' in', '', regex=True)
pk['bed_length'] = pk['bed_length'].astype('float')
pk.dropna(how='all', inplace=True)
pk.sample(10)

/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,bed,bed_length,cabin
7054,Short,67.1,Crew Cab
1339,NaN,78.9,NaN
6424,NaN,67.1,NaN
1509,NaN,67.4,NaN
659,NaN,76.3,NaN
7099,NaN,98.1,NaN
6952,NaN,67.1,NaN
866,NaN,69.9,NaN
7388,Short,78.8,Extended Cab
454,NaN,78.9,NaN


### Then car table

In [81]:
Car = cars_mini[["vin", "sp_id", "dealer_zip", "city", # seller info attributes
            "price", "year", "make_name", "model_name", "body_type", "maximum_seating", "interior_color", "exterior_color", "mileage",  # car main attributes
            "length", "width", "height", "wheelbase", "front_legroom", "back_legroom",  # car detailed attributes
            "engine_displacement", "engine_type", "transmission", "transmission_display", "wheel_system", "wheel_system_display", "horsepower", "power", "torque",  # power info
            "fuel_tank_volume", "fuel_type", "city_fuel_economy", "highway_fuel_economy",  # economic info
            # 'is_depreciated', 'seller_id', 'country', 'zip', 'power_rpm', 'pound_foot', 'torque_rpm'\
            "is_new", "listed_date", "main_picture_url", 
           "owner_count", "seller_rating", 
           "trim_name", 
            ]]
# remove spaces in string

Car['body_type'] = Car['body_type'].str.replace(r' / ', '_', regex=True)

# rename sp_id, dealer_zip, etc.
Car.rename(columns={'sp_id': 'seller_id', 'dealer_zip': 'zip', 'power': 'power_rpm', 'torque': 'torque_rpm'}, inplace=True)

Car.insert(1, 'is_depreciated', False)

Car.loc[Depreciation['frame_damaged'] == True, 'is_depreciated'] = True
Car.loc[Depreciation['has_accidents'] == True, 'is_depreciated'] = True
Car.loc[Depreciation['salvage'] == True, 'is_depreciated'] = True
Car.loc[Depreciation['isCab'] == True, 'is_depreciated'] = True
Car.loc[Depreciation['theft_title'] == True, 'is_depreciated'] = True

# insert country column
Car.insert(5, 'country', 'USA')

# change strings to int
Car['power_rpm'] = Car['power_rpm'].str.replace(r'.*@', '', regex=True)
Car['power_rpm'] = Car['power_rpm'].str.replace(r' RPM', '', regex=True)
Car['power_rpm'] = Car['power_rpm'].str.replace(r',', '', regex=True)
Car['power_rpm'] = Car['power_rpm'].fillna(0)

Car.insert(29, 'pound_foot', Car['torque_rpm'])

Car['torque_rpm'] = Car['torque_rpm'].str.replace(r'.*@', '', regex=True)
Car['torque_rpm'] = Car['torque_rpm'].str.replace(r' RPM', '', regex=True)
Car['torque_rpm'] = Car['torque_rpm'].str.replace(r',', '', regex=True)
Car['torque_rpm'] = Car['torque_rpm'].fillna(0)

Car['pound_foot'] = Car['pound_foot'].str.replace(r'lb-ft.*', '', regex=True)
Car['pound_foot'] = Car['pound_foot'].fillna(0)

Car['power_rpm'] = Car['power_rpm'].astype('int')
Car['pound_foot'] = Car['pound_foot'].astype('int')
Car['torque_rpm'] = Car['torque_rpm'].astype('int')

Car['seller_id'] = Car['seller_id'].astype('int')

# other string to numeric

Car['back_legroom'] = Car['back_legroom'].str.replace(r' in', '', regex=True)
Car['back_legroom'] = Car['back_legroom'].str.replace(r'--', '0', regex=True)
Car['back_legroom'] = Car['back_legroom'].fillna(0)
Car['back_legroom'] = Car['back_legroom'].astype('float')

Car['front_legroom'] = Car['front_legroom'].str.replace(r' in', '', regex=True)
Car['front_legroom'] = Car['front_legroom'].str.replace(r'--', '0', regex=True)
Car['front_legroom'] = Car['front_legroom'].fillna(0)
Car['front_legroom'] = Car['front_legroom'].astype('float')

Car['fuel_tank_volume'] = Car['fuel_tank_volume'].str.replace(r' gal', '', regex=True)
Car['fuel_tank_volume'] = Car['fuel_tank_volume'].str.replace(r'--', '0', regex=True)
Car['fuel_tank_volume'] = Car['fuel_tank_volume'].fillna(0)
Car['fuel_tank_volume'] = Car['fuel_tank_volume'].astype('float')

Car['height'] = Car['height'].str.replace(r' in', '', regex=True)
Car['height'] = Car['height'].str.replace(r'--', '0', regex=True)
Car['height'] = Car['height'].fillna(0)
Car['height'] = Car['height'].astype('float')

Car['length'] = Car['length'].str.replace(r' in', '', regex=True)
Car['length'] = Car['length'].str.replace(r'--', '0', regex=True)
Car['length'] = Car['length'].fillna(0)
Car['length'] = Car['length'].astype('float')

Car['maximum_seating'] = Car['maximum_seating'].str.replace(r' seats', '', regex=True)
Car['maximum_seating'] = Car['maximum_seating'].str.replace(r'--', '0', regex=True)
Car['maximum_seating'] = Car['maximum_seating'].fillna(0)
Car['maximum_seating'] = Car['maximum_seating'].astype('int')

Car['owner_count'] = Car['owner_count'].fillna(1)
Car['owner_count'] = Car['owner_count'].astype('int')

Car['wheelbase'] = Car['wheelbase'].str.replace(r' in', '', regex=True)
Car['wheelbase'] = Car['wheelbase'].str.replace(r'--', '0', regex=True)
Car['wheelbase'] = Car['wheelbase'].fillna(0)
Car['wheelbase'] = Car['wheelbase'].astype('float')

Car['width'] = Car['width'].str.replace(r' in', '', regex=True)
Car['width'] = Car['width'].str.replace(r'--', '0', regex=True)
Car['width'] = Car['width'].fillna(0)
Car['width'] = Car['width'].astype('float')


Car['engine_displacement'] = Car['engine_displacement'].astype('float')


Car.sample(2)

/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,vin,is_depreciated,seller_id,zip,city,country,price,year,make_name,model_name,body_type,maximum_seating,interior_color,exterior_color,mileage,length,width,height,wheelbase,front_legroom,back_legroom,engine_displacement,engine_type,transmission,transmission_display,wheel_system,wheel_system_display,horsepower,power_rpm,pound_foot,torque_rpm,fuel_tank_volume,fuel_type,city_fuel_economy,highway_fuel_economy,is_new,listed_date,main_picture_url,owner_count,seller_rating,trim_name
5352,JTND4RBE4L3091694,False,366905,48328,Waterford,USA,20271.0,2020,Toyota,Corolla Hatchback,Hatchback,5,Black,Blizzard Pearl,3.0,172.0,70.5,57.1,103.9,42.0,29.9,2000.0,I4,CVT,Continuously Variable Transmission,FWD,Front-Wheel Drive,168.0,6100,151,4000,13.2,Gasoline,32.0,41.0,True,2020-05-23,https://static.cargurus.com/images/forsale/202...,1,4.700000,SE FWD
2931,JTJBARBZ8F2017192,False,411406,6492,Wallingford,USA,25415.0,2015,Lexus,NX 200t,SUV_Crossover,5,Black,Black,48132.0,182.3,83.9,64.8,104.7,42.8,36.1,2000.0,I4,A,Automatic,AWD,All-Wheel Drive,235.0,5600,258,4000,15.9,Gasoline,22.0,27.0,False,2020-08-16,https://static.cargurus.com/images/forsale/202...,1,4.545455,F Sport AWD


### Next is user table

In [82]:
user = cars_mini[['sp_id', 'sp_name', 'franchise_dealer']]
user.rename(columns={'sp_id': 'id', 'sp_name': 'name', 'franchise_dealer': 'is_franchise_dealer'}, inplace=True)
user.insert(2, 'email', 'xxx@xxx.com')
user.insert(3, 'type', 'dealer')

user.loc[user['is_franchise_dealer'] != True, 'type'] = 'personal'
user['id'] = user['id'].astype('int')

user.sample(2)

/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,id,name,email,type,is_franchise_dealer
5689,158236,Hines Park Ford,xxx@xxx.com,dealer,True
1427,293986,Autoworld Kia,xxx@xxx.com,dealer,True


### export the dataframe to csv files

In [12]:
debug_path = '/tmp/ece651/debug/'
Car.to_csv(debug_path + 'car.csv')
Depreciation.to_csv(debug_path + 'depreciation_info.csv')
user.to_csv(debug_path + 'user.csv', index=False)
pickup_truck.to_csv(debug_path + 'pickup_truck.csv')

In [83]:
os.getcwd()

'/Users/lemingqiu/Desktop/ECE651/Project'

In [84]:
cwd = os.getcwd()
Car.to_csv(cwd + '/car.csv')
Depreciation.to_csv(cwd + '/depreciation_info.csv')
user.to_csv(cwd + '/user.csv', index=False)
pickup_truck.to_csv(cwd + '/pickup_truck.csv')